In [1]:
!pip install pymysql
!pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import time
import pandas as pd
import numpy as np
import datetime
import pymysql
from sqlalchemy import create_engine
import FinanceDataReader as fdr
from tqdm import tqdm

In [3]:
code_data = pd.read_csv('./raw_data.csv', index_col = 0)

code_date['상장일'] dtype을 object → datetime 변경

In [4]:
code_data['상장일'] = pd.to_datetime(code_data['상장일'])
code_data['종목코드'] = code_data['종목코드'].apply(lambda x: str(x).zfill(6))

In [5]:
code_data[:5]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,000210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,004840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,078930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,GS글로벌,001250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시


# 과제 1

데이터를 dataframe에서 list로 변환

In [7]:
lst_stock = code_data.values.tolist()

dictionary 사용

In [8]:
dic_code2company = {}
for row in lst_stock:
    company, code, date = row[0], row[1], row[4]
    if date <= datetime.datetime(2018, 1, 2):
        dic_code2company[code] = company

In [9]:
OF = open('assignment1_sql.txt', 'w', encoding = 'utf-8')
for code in dic_code2company.keys():
    data = '{}\t{}\n'.format(code, dic_code2company[code])
    OF.write(data)
OF.close()

# 과제 2

In [10]:
db_dsml = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'stock_user', 
    passwd = 'bigdata', 
    db = 'stock', 
    charset = 'utf8'
)

cursor = db_dsml.cursor()

OperationalError: (1049, "Unknown database 'stock'")

In [11]:
dic_code2date = {}

OF = open('assignment2_sql.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2company.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2018-01-01' AND '2021-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for row in lst_stock:
        if row[4] * row[5] >= 100000000000:
            data = '{}\t{}\n'.format(code, row[0].strftime('%Y%m%d'))
            OF.write(data)
            if code not in dic_code2date.keys():
                dic_code2date[code] = []
                dic_code2date[code].append(row[0])
            else:
                dic_code2date[code].append(row[0])
OF.close()

100%|█████████████████████████████████████████| 1999/1999 [00:31<00:00, 63.38it/s]


# 과제 3

In [11]:
OF = open('assignment3_sql.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2018-01-01' AND '2020-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for i, row_lst_stock in enumerate(lst_stock):
        # 예외 처리
        if (i < 9) or (i >= len(lst_stock)-1):
            continue  
        date = row_lst_stock[0]
        if date not in dic_code2date[code]:
            continue
        
        # 11 days data
        sub_stock = lst_stock[i-9:i+1]
        lst_data = []
        for row_sub_stock in sub_stock:
            open, high, low, close, volume = row_sub_stock[1:6]
            trading_value = close * volume
            lst_data += [open, high, low, close, trading_value]
            del open
        data = ','.join(map(str, lst_data))
        
        # label
        label = int(lst_stock[i+1][-1] >= 0.02)
        
        result = '{}\t{}\t{}\t{}\n'.format(code, date.strftime("%Y%m%d"), data, label)
        OF.write(result)
OF.close()

100%|██████████| 1134/1134 [00:15<00:00, 71.16it/s]


# 과제 3-2

In [12]:
OF = open('assignment3-2_sql.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date
                BETWEEN '2021-01-01' AND '2021-06-30'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for i, row_lst_stock in enumerate(lst_stock):
        # 예외 처리
        if (i < 9) or (i >= len(lst_stock)-1):
            continue  
        date = row_lst_stock[0]
        if date not in dic_code2date[code]:
            continue
        
        # 11 days data
        sub_stock = lst_stock[i-9:i+1]
        lst_data = []
        for row_sub_stock in sub_stock:
            open, high, low, close, volume = row_sub_stock[1:6]
            trading_value = close * volume
            lst_data += [open, high, low, close, trading_value]
            del open
        data = ','.join(map(str, lst_data))
        
        # label
        label = int(lst_stock[i+1][-1] >= 0.02)
        
        result = '{}\t{}\t{}\t{}\n'.format(code, date.strftime("%Y%m%d"), data, label)
        OF.write(result)
OF.close()

100%|██████████| 1134/1134 [00:04<00:00, 246.06it/s]
